# 10 ML Pipeline

**Project:** NORI  
**Author:** Yuseof J  
**Date:** 22/12/25

### **Purpose**
Load the raw NY State tract shapefile, filter for NYC tracts, and set crs to Long Island/NYC. 

### **Inputs**
- `data/raw/tiger_tracts_ny/tl_2025_36_tract.shp`

### **Outputs**
- `data/processed/nyc_tracts.gpkg`
  
--------------------------------------------------------------------------

### 0. Imports and Setup

In [15]:
# package imports
import os
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GroupKFold

# specify filepaths
path_model_features = 'data/processed/master_model_features.csv'
path_model_targets = 'data/processed/master_model_targets.csv'

# ensure cwd is project root for file paths to function properly
project_root = Path(os.getcwd())            # get current directory
while not (project_root / "data").exists(): # keep moving up until in parent
    project_root = project_root.parent
os.chdir(project_root)                      # switch to parent directory

### 1. Load Data

In [2]:
# features (X), targets (y)
X = pd.read_csv(path_model_features)
y = pd.read_csv(path_model_targets)

### 2. Prepare Data

For this first pass, we'll be focusing on just one target to get the pipeline up and running

In [6]:
TARGET = "CASTHMA_CrudePrev"

y = y[['GEOID', TARGET]]

In [11]:
# take note of feature columns
feature_cols = X.columns.tolist()

# select only tracts with a value for the target
df_model = y.merge(X, how="left", on="GEOID")

# reseperate features and target for modeling
X = df_model[feature_cols]
y = df_model[TARGET]

Spatial CV

Here we'll perform a basic spatial split in the data for cross-validation. It's important to split the data by some meaningful geographic grouping because the tracts are not independent of one another: neighboring tracts share characteristics like socioeconomic status,, health outcomes, air quality, and green coverage. If we split the spatial data at random, we may end up with the scenario where one tract is used for training, and its neighbor is used in testing. This is a problem because those two tracts are too similar, meaning the model isn't learning to generalize to new places, but rather memorizing information about similar tract contexts/characteristics. This will lead to overestimated model performance. 

In [41]:
# split data by geographic clusters

# bring the tract gdf in to get geometry
# use centroid for clustering
# assign cluster ID 
# join cluster ID to df_model on GEOID

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,029,008400,36029008400,1400000US36029008400,84,Census Tract 84,G5020,S,10966624,3505091,+42.9713848,-078.9194986,"POLYGON ((-78.94456 42.98506, -78.94216 42.992..."
1,36,103,123600,36103123600,1400000US36103123600,1236,Census Tract 1236,G5020,S,2302367,1082191,+40.6608399,-073.4145754,"POLYGON ((-73.42559 40.65629, -73.42529 40.656..."
2,36,103,146001,36103146001,1400000US36103146001,1460.01,Census Tract 1460.01,G5020,S,2225464,0,+40.7703277,-073.2532537,"POLYGON ((-73.26159 40.76307, -73.2615 40.7636..."
3,36,103,190402,36103190402,1400000US36103190402,1904.02,Census Tract 1904.02,G5020,S,44073411,23956,+40.8468673,-072.6336641,"POLYGON ((-72.72668 40.8339, -72.72515 40.8387..."
4,36,103,158709,36103158709,1400000US36103158709,1587.09,Census Tract 1587.09,G5020,S,13099359,110761,+40.8517499,-072.9216255,"POLYGON ((-72.94716 40.8556, -72.94649 40.8576..."


### 3. Data Processing and Filtering

In [42]:
# filter for nyc boroughs
gdf_tracts_nyc = gdf_tracts[gdf_tracts.COUNTYFP.isin(nyc_fips)]

print("Total tracts: ", len(gdf_tracts_nyc))

Total tracts:  2327


In [43]:
# set coordinate reference system 
gdf_tracts_nyc = gdf_tracts_nyc.to_crs(nyc_crs)

### 4. Save Data

In [44]:
# export processed tract data
gdf_tracts_nyc.to_file(path_output_processed_geodata, layer="tracts")